In [4]:
import tensorflow as tf

batch_size = 64
boxes = tf.zeros((batch_size,4,2))

print("1단계 연산 준비 : ",boxes.shape)

first_linear = tf.keras.layers.Dense(units=1,use_bias=False)

first_out = first_linear(boxes)
first_out = tf.squeeze(first_out,axis=-1)

print("1단계 연산 결과 : ",first_out.shape)
print("1단계 linear layer의 weight형태 : ",first_linear.weights[0].shape)

print("2단계 연산 준비 : ",first_out.shape)

second_linear = tf.keras.layers.Dense(units=1,use_bias = False)
second_out = second_linear(first_out)
second_out = tf.squeeze(second_out,axis=-1)

print("2단계 연산 결과 : ",second_out.shape)
print("2단계 linear layer의 weight형태 : ",second_linear.weights[0].shape)

1단계 연산 준비 :  (64, 4, 2)
1단계 연산 결과 :  (64, 4)
1단계 linear layer의 weight형태 :  (2, 1)
2단계 연산 준비 :  (64, 4)
2단계 연산 결과 :  (64,)
2단계 linear layer의 weight형태 :  (4, 1)


In [11]:
import tensorflow as tf

batch_size = 64
boxes = tf.zeros((batch_size, 4, 2))

print("1단계 연산 준비:", boxes.shape)

########
# Step 1: (4,2)차원인 boxes를 (4,3)으로 확장시키는 Linear Layer를 활용하세요.
# Enter Your Source
print("1단계 연산 준비 : ",boxes.shape)

first_linear = tf.keras.layers.Dense(units=3,use_bias=False)

first_out = first_linear(boxes)
first_out = tf.expand_dims(first_out,1)


########

print("1단계 연산 결과:", first_out.shape)
print("1단계 Linear Layer의 Weight 형태:", first_linear.weights[0].shape)
print("\n2단계 연산 준비:", first_out.shape)

# Dense = Linear
second_linear = tf.keras.layers.Dense(units=1, use_bias=False)
second_out = second_linear(first_out)
second_out = tf.squeeze(second_out, axis=-1)

print("2단계 연산 결과:", second_out.shape)
print("2단계 Linear Layer의 Weight 형태:", second_linear.weights[0].shape)

print("\n3단계 연산 준비:", second_out.shape)

########
# Step 2: 4차원인 second_out을 하나의 실수으로 집약시키는 Linear Layer를 활용하세요.
# Enter Your Source
third_linear = tf.keras.layers.Dense(units=1, use_bias=False)
third_out = third_linear(second_out)
third_out = tf.squeeze(third_out, axis=-1)

print("3단계 연산 결과:", third_out.shape)
print("3단계 Linear Layer의 Weight 형태:", third_linear.weights[0].shape)

########
# Step 3: 모든 params를 더하여 total_parmams를 구하세요.
# Enter Your Source
total_params = \
first_linear.count_params() + \
second_linear.count_params() + \
third_linear.count_params()

print(total_params)

########

print("총 Parameters:", total_params)

1단계 연산 준비: (64, 4, 2)
1단계 연산 준비 :  (64, 4, 2)
1단계 연산 결과: (64, 1, 4, 3)
1단계 Linear Layer의 Weight 형태: (2, 3)

2단계 연산 준비: (64, 1, 4, 3)
2단계 연산 결과: (64, 1, 4)
2단계 Linear Layer의 Weight 형태: (3, 1)

3단계 연산 준비: (64, 1, 4)
3단계 연산 결과: (64, 1)
3단계 Linear Layer의 Weight 형태: (4, 1)
13
총 Parameters: 13


In [21]:
import numpy as np
from tensorflow.keras.layers import Input,Dense,Conv2D,MaxPooling2D,UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
import json
import matplotlib.pyplot as plt 

(x_train,_),(x_test,_) = mnist.load_data()
print(x_train.shape)
x_train = np.expand_dims(x_train,axis=3)
x_test = np.expand_dims(x_test,axis=3)

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

(60000, 28, 28)


In [22]:
input_shape = x_train.shape[1:]
input_img = Input(shape=input_shape)

encode_conv_layer_1 = Conv2D(16,(3,3),activation='relu',padding='same')
encode_pool_layer_1 = MaxPooling2D((2,2),padding='same')
encode_conv_layer_2 = Conv2D(8,(3,3),activation='relu',padding='same')
encode_pool_layer_2 = MaxPooling2D((2,2),padding='same')
encode_conv_layer_3 = Conv2D(4,(3,3),activation='relu',padding='same')
encode_pool_layer_3 = MaxPooling2D((2,2),padding='same')

encoded = encode_conv_layer_1(input_img)
encoded = encode_pool_layer_1(encoded)
encoded = encode_conv_layer_2(encoded)
encoded = encode_pool_layer_2(encoded)
encoded = encode_conv_layer_3(encoded)
encoded = encode_pool_layer_3(encoded)

decode_conv_layer_1 = Conv2D(4,(3,3),activation='relu',padding='same')
decode_upsample_layer_1 = UpSampling2D((2,2))
decode_conv_layer_2 = Conv2D(8,(3,3),activation='relu',padding='same')
decode_upsample_layer_2 = UpSampling2D((2,2))
decode_conv_layer_3 = Conv2D(16,(3,3),activation='relu',padding='same')
decode_upsample_layer_3 = UpSampling2D((2,2))
decode_conv_layer_4 = Conv2D(1,(3,3),activation='sigmoid',padding='same')

decoded = decode_conv_layer_1(encoded)
decoded = decode_upsample_layer_1(decoded)
decoded = decode_conv_layer_2(decoded)
decoded = decode_upsample_layer_2(decoded)
decoded = decode_conv_layer_3(decoded)
decoded = decode_upsample_layer_3(decoded)
decoded = decode_conv_layer_4(decoded)

autoencoder = Model(input_img,decoded)
autoencoder.summary()



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 4)           292       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 4)           0     

In [34]:
import tensorflow as tf

batch_size = 64
boxes = tf.zeros((batch_size, 4, 2))     # Tensorflow는 Batch를 기반으로 동작하기에,
                                         # 우리는 사각형 2개 세트를 batch_size개만큼
                                         # 만든 후 처리를 하게 됩니다.
#print(boxes)
print("1단계 연산 준비:", boxes.shape)

first_linear = tf.keras.layers.Dense(units=1, use_bias=False) 
# units은 출력 차원 수를 의미합니다.
# Weight 행렬 속 실수를 인간의 뇌 속 하나의 뉴런 '유닛' 취급을 하는 거죠!

first_out = first_linear(boxes)
first_out = tf.squeeze(first_out, axis=-1) # (4, 1)을 (4,)로 변환해줍니다.
                                           # (불필요한 차원 축소)

print("1단계 연산 결과:", first_out.shape)
print("1단계 Linear Layer의 Weight 형태:", first_linear.weights[0].shape)

print("\n2단계 연산 준비:", first_out.shape)

second_linear = tf.keras.layers.Dense(units=1, use_bias=False)
second_out = second_linear(first_out)
second_out = tf.squeeze(second_out, axis=-1)
print(second_out)
print("2단계 연산 결과:", second_out.shape)
print("2단계 Linear Layer의 Weight 형태:", second_linear.weights[0].shape)

1단계 연산 준비: (64, 4, 2)
1단계 연산 결과: (64, 4)
1단계 Linear Layer의 Weight 형태: (2, 1)

2단계 연산 준비: (64, 4)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(64,), dtype=float32)
2단계 연산 결과: (64,)
2단계 Linear Layer의 Weight 형태: (4, 1)


In [40]:
import tensorflow as tf
batch_size = 64
boxes = tf.zeros((batch_size,4,2))

print("1단계 연산 준비 : ",boxes.shape)
first_linear = tf.keras.layers.Dense(units=3, use_bias=False)
first_out = first_linear(boxes)
print("\n2단계 연산 준비:", first_out.shape)

# Dense = Linear
second_linear = tf.keras.layers.Dense(units=1, use_bias=False)
second_out = second_linear(first_out)
second_out = tf.squeeze(second_out, axis=-1)

print("2단계 연산 결과:", second_out.shape)
print("2단계 Linear Layer의 Weight 형태:", second_linear.weights[0].shape)

print("\n3단계 연산 준비:", second_out.shape)

1단계 연산 준비 :  (64, 4, 2)

2단계 연산 준비: (64, 4, 3)
2단계 연산 결과: (64, 4)
2단계 Linear Layer의 Weight 형태: (3, 1)

3단계 연산 준비: (64, 4)
